In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

# Define the WideResNet model (simplified)
class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, drop_rate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.drop_rate = drop_rate
        self.equal_in_out = (in_planes == out_planes)
        self.shortcut = (not self.equal_in_out) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, padding=0, bias=False) or None

    def forward(self, x):
        if not self.equal_in_out:
            x = F.relu(self.bn1(x), inplace=True)
            out = self.conv1(x)
        else:
            out = F.relu(self.bn1(x), inplace=True)
            out = self.conv1(out)
        out = F.relu(self.bn2(out), inplace=True)
        if self.drop_rate > 0:
            out = F.dropout(out, p=self.drop_rate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equal_in_out else self.shortcut(x), out)

class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, drop_rate=0.0):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, out_planes, nb_layers, stride, drop_rate)

    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, drop_rate):
        layers = []
        for i in range(nb_layers):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes, i == 0 and stride or 1, drop_rate))
        return nn.Sequential(*layers)

    def forward(self, x):
        return self.layer(x)

class WideResNet(nn.Module):
    def __init__(self, depth, num_classes, widen_factor=10, drop_rate=0.0):
        super(WideResNet, self).__init__()
        nChannels = [16, 16 * widen_factor, 32 * widen_factor, 64 * widen_factor]
        assert ((depth - 4) % 6 == 0)
        n = (depth - 4) // 6
        block = BasicBlock
        self.conv1 = nn.Conv2d(3, nChannels[0], kernel_size=3, stride=1, padding=1, bias=False)
        self.block1 = NetworkBlock(n, nChannels[0], nChannels[1], block, 1, drop_rate)
        self.block2 = NetworkBlock(n, nChannels[1], nChannels[2], block, 2, drop_rate)
        self.block3 = NetworkBlock(n, nChannels[2], nChannels[3], block, 2, drop_rate)
        self.bn1 = nn.BatchNorm2d(nChannels[3])
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(nChannels[3], num_classes)
        self.nChannels = nChannels[3]

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = F.relu(self.bn1(out), inplace=True)
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.nChannels)
        return self.fc(out)

# CIFAR-100 dataset
def get_cifar100_loaders(batch_size=128):
    transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.507, 0.486, 0.441), (0.267, 0.256, 0.276))])

    train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

# Function to calculate test loss and accuracy
def evaluate_model(model, test_loader, criterion):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.cuda(), targets.cuda()

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    avg_loss = test_loss / total
    accuracy = 100. * correct / total

    return avg_loss, accuracy

# Training with multiple parameter groups
def train_wideresnet_28_10():

    # Initialize model, loss, and optimizer
    model = WideResNet(depth=28, num_classes=100, widen_factor=10, drop_rate=0.3)
    model = model.cuda()

    # Define optimizer with multiple parameter groups
    optimizer = optim.Adam([
        {'params': model.conv1.parameters(), 'lr': 0.001, 'weight_decay': 1e-4},  # Learning rate for initial conv layer
        {'params': model.block1.parameters(), 'lr': 0.001, 'weight_decay': 1e-4}, # Learning rate for 1st block
        {'params': model.block2.parameters(), 'lr': 0.001, 'weight_decay': 1e-4}, # Learning rate for 2nd block
        {'params': model.block3.parameters(), 'lr': 0.001, 'weight_decay': 1e-4}, # Learning rate for 3rd block
        {'params': model.fc.parameters(), 'lr': 0.01, 'weight_decay': 1e-3}  # Separate LR for fully connected
    ])

    # Scheduler to reduce LR every 30 epochs
    scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

    # Training loop
    num_epochs = 100
    train_loader, test_loader = get_cifar100_loaders()
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.cuda(), targets.cuda()

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(train_loader):.4f}')

        # Evaluate the model on test data after each epoch
        test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

        # Adjust learning rate
        scheduler.step()

    print('Finished Training')

if __name__ == '__main__':
    train_wideresnet_28_10()


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/100], Training Loss: 4.3176
Epoch [1/100], Test Loss: 4.0591, Test Accuracy: 6.80%
Epoch [2/100], Training Loss: 3.8763
Epoch [2/100], Test Loss: 3.8001, Test Accuracy: 9.60%
Epoch [3/100], Training Loss: 3.5818
Epoch [3/100], Test Loss: 3.4422, Test Accuracy: 16.06%
Epoch [4/100], Training Loss: 3.2161
Epoch [4/100], Test Loss: 3.0592, Test Accuracy: 22.22%
Epoch [5/100], Training Loss: 2.8770
Epoch [5/100], Test Loss: 2.7916, Test Accuracy: 27.21%
Epoch [6/100], Training Loss: 2.6094
Epoch [6/100], Test Loss: 2.6047, Test Accuracy: 32.65%
Epoch [7/100], Training Loss: 2.3971
Epoch [7/100], Test Loss: 2.6177, Test Accuracy: 33.53%
Epoch [8/100], Training Loss: 2.2058
Epoch [8/100], Test Loss: 2.5009, Test Accuracy: 36.54%
Epoch [9/100], Training Loss: 2.1568
Epoch [9/100], Test Loss: 2.0443, Test Accuracy: 44.43%
Epoch [10/100], Training Loss: 1.9595
Epoch [10/100], Test Loss: 2.0438, Test Accuracy: 4